### Project: INM713-coursework
**Author**: Zac Detorakis (zacharias.detorakis@city.ac.uk)

**Version**: 1.0<br>
**Date**: 15/03/2021

In [39]:
# Import Libraries
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
# from rdflib.util import guess_format
import pandas as pd
import math
# # from stringcmp import isub
# # from lookup import DBpediaLookup
# import csv

# import owlrl


In [9]:
class FinalCoursework(object):
    """
    This will contain the solution for the coursework for the SW&KGT
    """
    
    def __init__(self, input_csv):
        
        # The file containing the data to load in the KG
        self.file = input_csv
        
        #Intialise the KG
        self.g = Graph()
        
        #read the raw data into a dataframe
        self.df = pd.read_csv(filepath_or_buffer = self.file, sep=',', quotechar='"',escapechar="\\")
        
if __name__ == '__main__':
#     input_csv = "INM713_coursework_data_pizza_8358_1_reduced - small.csv"
    input_csv = "INM713_coursework_data_pizza_8358_1_reduced.csv"
    
    solution = FinalCoursework(input_csv)
    
        

First of we need to perform some exploratory data analysis before we start loading the data to ensure the data consistency and perhaps perform a bit of data cleansing.
We will start with the address part as that is the easier one to profile.

Given that the addresses are unique per restaurant and then repeated for all menu items of that restaurant we will get a subsection of the df to better assess the extent of problematic records that need fixing. As per our ontology, the address properties are:
* address (i.e. representing the address line)
* city
* country
* state and
* postcode

In [33]:
# Construct the address dataframe with the columns related to the restaurant addresses. 
# We'll keep the name column in there just for reference so we can count restaurant with problematic addresses

df_address = solution.df[['name','address','city','country','state','postcode']]
print(f"There are a total of: {len(df_address)} restaurant/menu_items")

# remove duplicate rows
df_address = df_address.drop_duplicates()
print(f"There are a total of: {len(df_address)} restaurants")

There are a total of: 3510 restaurant/menu_items
There are a total of: 991 restaurants


In [35]:
# Check the country column
df_address['country'].value_counts()

US    991
Name: country, dtype: int64

looks like the country colunn is the specific file is always populated with the country US and doesn't need any cleansing

In [42]:
# Check the state column
print(f'There are {len(df_address["state"].unique())} unique values for state')
print(f'There are {round((len(df_address[df_address["state"].str.len()>2])/len(df_address))*100,2)}% where the state is not 2 characters long')
print(f'There are {round((len(df_address[df_address["state"].str.len()==2])/len(df_address))*100,2)}% where the state is exactly 2 characters long')

There are 281 unique values for state
There are 29.97% where the state is not 2 characters long
There are 70.03% where the state is exactly 2 characters long


The above results indicate that for 30% of restaurants we may need to fix the state column

In [43]:
# https://solidproject.org/developers/vocabularies/well-known

In [44]:
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?country ?capital
WHERE {
 ?x a dbo:Country.
 ?x dbo:longName ?country.
 ?x dbp:capital ?capital
}

SyntaxError: invalid syntax (<ipython-input-44-4c6d0f15607c>, line 1)

In [71]:
g = Graph()

qres = g.query(
"""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?x
WHERE {
 ?x a dbo:City.
}
""")

#Single row with one boolean vale
for row in qres:
    print(f"{str(row.x)},")

In [59]:
from lookup import DBpediaLookup

In [64]:
def getExternalKGURI(name):
    '''
    Approximate solution: We get the entity with highest lexical similarity
    The use of context may be necessary in some cases        
    '''
    
    dbpedia = DBpediaLookup()
    entities = dbpedia.getKGEntities(name, 1)

#     print(current_uri)
    return entities[0].ident

In [72]:
g = Graph()

dpo = Namespace("http://dbpedia.org/resource/")
g.bind("dpo", dpo)


getExternalKGURI('Omaha')

'http://dbpedia.org/resource/Omaha,_Nebraska'

In [68]:
kg_entity = KGEntity()

NameError: name 'KGEntity' is not defined